  - 단일변수 회귀
  - 다변수 회귀분석

앙상블
  - 의사결정 트리
  - 랜덤포레스트
  - 스태킹
  - 베깅
  - 부트팅

In [ ]:
from sklearn.ensemble import StackingRegressor

In [ ]:
'''
개별 모델이 예측한 데이터를 기반으로
회귀모델을 여러개 만들고 각각 학습
필요하면 하이퍼파라메터도 튜닝
교차검증을 통해서 각 모델별 성능점수를 추출해서 상위 5개 이런식으로
모델을리스트로 만들어서 -- estimators

estimators = [
  ('randomforest', rf),
  ('extrtreegreg', ext),
  ('xgboost', xgb)
]

 StackingRegressor(estimators, final_estimator = xgb)

'''

데이콘 데이터 로드

In [7]:
pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 28.5 MB/s eta 0:00:00


In [17]:
import random
import os

import numpy as np
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from rdkit import DataStructs
from rdkit.Chem import PandasTools, AllChem

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [9]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42) # Seed 고정

In [3]:
!unzip '/content/drive/MyDrive/데이콘_신약개발/데이콘 신약데이터.zip'

Archive:  /content/drive/MyDrive/데이콘_신약개발/데이콘 신약데이터.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [10]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [11]:
PandasTools.AddMoleculeColumnToFrame(train,'SMILES','Molecule')
PandasTools.AddMoleculeColumnToFrame(test,'SMILES','Molecule')

In [13]:
def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

In [14]:
# FPs column 추가
train["FPs"] = train.Molecule.apply(mol2fp)
test["FPs"] = test.Molecule.apply(mol2fp)

In [15]:
# 사용할 column만 추출
train = train[['FPs','MLM', 'HLM']]
test = test[['FPs']]

In [19]:
class CustomDataset(Dataset):
    def __init__(self, df, target, transform, is_test=False):
        self.df = df
        self.target = target # HLM or MLM
        self.is_test = is_test # train,valid / test

        self.feature_select = transform
        if not self.is_test:
            self.fp = self.feature_select.fit_transform(np.stack(df['FPs']))
        else: # valid or test
            self.fp = self.feature_select.transform(np.stack(df['FPs']))

    def __getitem__(self, index):
        fp = self.fp[index]
        if not self.is_test: # test가 아닌 경우(label 존재)
            label = self.df[self.target][index]
            return torch.tensor(fp).float(), torch.tensor(label).float().unsqueeze(dim=-1) # feature, label

        else: # test인 경우
            return torch.tensor(fp).float() # feature

    def __len__(self):
        return len(self.df)

In [20]:
transform = VarianceThreshold(threshold=0.05)

train_MLM = CustomDataset(df=train, target='MLM', transform=transform, is_test=False)
train_HLM = CustomDataset(df=train, target='HLM', transform=transform, is_test=False)

input_size = train_MLM.fp.shape[1]
input_size

251

In [21]:
# train,valid split
train_MLM_dataset, valid_MLM_dataset = train_test_split(train_MLM, test_size=0.2, random_state=42)
train_HLM_dataset, valid_HLM_dataset = train_test_split(train_HLM, test_size=0.2, random_state=42)

In [44]:
# 예측모델 후보 - 랜덤포레스트 회귀
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()


In [ ]:
import tqdm
for train_data,target in tqdm.tqdm( train_MLM_dataset):
  rfr.fit(train_data.numpy().reshape(1,-1), target.numpy())

 27%|██▋       | 766/2798 [01:53<04:07,  8.22it/s]